<a href="https://colab.research.google.com/github/arsalanyaghoobi/Full-Course-on-Bert-and-Transformers/blob/main/Test_for_Bert_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import tensorflow as tf     # TensorFlow is a library which can be applied to all the machine learning algorithms especially deep learning with neural network.
device_name= tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [45]:
!pip install transformers tensorflow_datasets     # Provides thousands of pretrained models
                                                  # provides APIs to quickly download and use those pretrained models
                                                  #Transformers is backed by the two most popular deep learning libraries, PyTorch and TensorFlow

In [46]:
import tensorflow_datasets as tfds
(ds_train, ds_test),ds_info=tfds.load("imdb_reviews",#tfds.load
                                    split=(tfds.Split.TRAIN,tfds.Split.TEST),
                                    as_supervised=True,
                                    with_info=True)
print(ds_train,ds_test,ds_info)

INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load dataset info from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
INFO:absl:Reusing dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split (Split('train'), Split('test')), from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0


<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)> <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)> tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly

1)Positional arguments must appear before a keyword argument in Python. This is because Python interprets positional arguments in the order in which they appear. Then, it interprets the keyword arguments that have been specified.


In [47]:
for review, label in tfds.as_numpy(ds_train.take(100)): ##### not neccesary step
  print("review", review[0:5],"Label", label)

review b'This ' Label 0
review b'I hav' Label 0
review b'Mann ' Label 0
review b'This ' Label 1
review b'As ot' Label 1
review b'This ' Label 1
review b'Okay,' Label 0
review b'The f' Label 0
review b'I rea' Label 0
review b'Sure,' Label 0
review b'Durin' Label 0
review b'Cute ' Label 1
review b'This ' Label 1
review b'Put t' Label 0
review b'Hilar' Label 1
review b'It wa' Label 0
review b'This ' Label 1
review b'Final' Label 1
review b'India' Label 1
review b'Natha' Label 0
review b'I can' Label 1
review b'In a ' Label 1
review b'A pre' Label 1
review b'Excep' Label 1
review b'In th' Label 1
review b'Well,' Label 0
review b'I act' Label 0
review b'I lik' Label 0
review b'My ma' Label 1
review b'This ' Label 0
review b'Mike ' Label 0
review b'(Hone' Label 0
review b'The "' Label 0
review b'I was' Label 0
review b"I'm s" Label 0
review b'A mov' Label 1
review b'Of th' Label 1
review b'"Wild' Label 0
review b'This ' Label 0
review b'This ' Label 1
review b'I hav' Label 0
review b'Altho' 

1)In the section above we called for review and label.

2)we used `tfds.as_numpy()` in order to turn the data into numpy arrays.

In [48]:
from transformers import BertTokenizer        #import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')    #choose the type of tokenizer(https://huggingface.co/transformers/pretrained_models.html)

In order to use BERT text embeddings as input to train text classification model, we need to tokenize our text reviews. Tokenization refers to dividing a sentence into individual words.
The BERT tokenizer use WordPiece vocabulary.It has over 30000 words and it maps pretrained embeddings for each.Each word has its own IDs; we would need to map the tokens to those IDs.

In [49]:
vocabulary= tokenizer.get_vocab()       # tokenize our text reviews; why dataset name is not mentioned in the paranthesis?
vocabulary_list=list(vocabulary.keys())
print(vocabulary_list[4000])
print(len(vocabulary))

tears
30522


In order to be able to enlist the vocabularies and make them all callable,
`vocab_list=list(vocabulary.keys())`
is used.

In [50]:
max_length_test= 50
test_sentence='test tokenization sentence by Arsalan.Followed by another sentence'

`Becareful this section on , you only will experinece processing your test data and not the training.`

Adding Special Token (CLS tokens are always at the beginning of the sentence and SEP tokens always at the end).
how an input sentence should be represented in BERT. BERT embeddings are trained with two training tasks:

Classification Task: to determine which category the input sentence should fall into
Next Sentence Prediction Task: to determine if the second sentence naturally follows the first sentence.

Sum Up:`to preprocess the input text data, the first thing we will have to do is to add the [CLS] token at the beginning, and the [SEP] token at the end of each input text.`

In [51]:
test_sentence_with_special_tokens='[CLS]'+ test_sentence +'[SEP]'
tokenized_test_sentence_with_special_tokens=tokenizer.tokenize(test_sentence_with_special_tokens)
print(tokenized_test_sentence_with_special_tokens)

['[CLS]', 'test', 'token', '##ization', 'sentence', 'by', 'ars', '##ala', '##n', '.', 'followed', 'by', 'another', 'sentence', '[SEP]']




Convert Tokens to IDs in WordPiece

In [52]:
input_ids = tokenizer.convert_tokens_to_ids(tokenized_test_sentence_with_special_tokens)
print(input_ids)
print(len(input_ids))

[101, 3231, 19204, 3989, 6251, 2011, 29393, 7911, 2078, 1012, 2628, 2011, 2178, 6251, 102]
15


Every time you have an input ID a attentionmask is required.The attention mask is a binary tensor `indicating the position of the padded indices` so that the model does not attend to them. For the BertTokenizer, **1 indicates a value that should be attended to, while 0 indicates a padded value**. This attention mask is in the dictionary returned by the tokenizer under the key “attention_mask”

In [53]:
attention_mask = [1] * len(input_ids)    #why should we have attention mask? Attention mask is required after the pooling!!!! the answer is clear: we only calculate the mask value ...
print(attention_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [54]:
padding_length = max_length_test - len(input_ids)  # calculating pad length
print(padding_length)
input_ids= input_ids + ([0] * padding_length)      # input ID = input ID +(Number of pad characters * [0])
print(input_ids)

35
[101, 3231, 19204, 3989, 6251, 2011, 29393, 7911, 2078, 1012, 2628, 2011, 2178, 6251, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Because you updated your input_ID it is required to update you attention mask as well;accordingly you wont pay attention to your padded tokens.

In [55]:
attention_mask=(attention_mask + [0]*padding_length) #attention mask as been calculated above = [1]* len(input_ids)
                                                     # at this satge we want to impose it on ids, so : [1]*len(input_ids) + [0]*padding_length

In the next step `token_type_ids` will differentiate consequent sequences from eachother by identifying IDs of a sequent by 0 and the next one by 1.

In [56]:
token_type_ids =[0]*max_length_test  # bcause the largest sequence of our dataset is max_length, thus ,in order to differentiate first and second sequences from each other we multiplied
                                     # max_length into [0] and [1], so that we will access to both. On the other hand if you only need one of the sentences do not multiply by [1].

it is used for defining the kind of input including distinguishing question from answer, distinguishing causes and effects.

defining a bert input, it is needed to define three elements:
1)Token ids
2)Token Type ids
3)attention mask

DONT FORGET that dataset gets into bert token by token. 


`BEFORE ENCODING IT IS NEEDED TO MAKE A DICTIONARY LIST`




In [57]:
bert_input={
    "token_ids":input_ids,
    "token_type_ids":token_type_ids,
    "attention_mask":attention_mask
}
print(bert_input)       # The required items to define Bert input features are token_ids, token_type_ids and attention_mask (Attention : data is converted into dictiionary)

{'token_ids': [101, 3231, 19204, 3989, 6251, 2011, 29393, 7911, 2078, 1012, 2628, 2011, 2178, 6251, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In the sections above, we received the data as sentence and then we turned it into tokenz;we maped the tokens to corresponding IDs using(tokenizer and convert_token_to_ID);in the section below, we use encode plus as a substitute for all mentioned above.This method adds special tokens auomatically.




In [58]:
bert_input= tokenizer.encode_plus(
    test_sentence,
    max_length_test=max_length_test,
    add_special_tokens=True,
    pad_to_max_length=True,
    return_attention_mask=True
)
print(bert_input)         ##### Optional 

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Keyword arguments {'max_length_test': 50} not recognized.


{'input_ids': [101, 3231, 19204, 3989, 6251, 2011, 29393, 7911, 2078, 1012, 2628, 2011, 2178, 6251, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


`DONT FORGET , THE INPUT TO ENCODE_PLUS WAS ONLY YOUR TEST DATASET AND NOT TRAINING.`

**Input IDs:** The input IDs are often the only required parameters to be passed to the model as input.They are token indices, numerical representations of tokens building the sequences that will be used as input by the model.


**Attention Mask:** using Mask inorder to avoid attention on the **Pad Token Indices**. Mask values are selected in a range of [0,1]; 1 for tokens that are not masked and 0 for masked tokens.


**Token Type IDs:**Some models purpose is to do sequence classification or question answering.These required two different sequences to be encoded in the same input IDs.They are usually seperated by special tokens, such as classifiers and separator tokens.for example the bert model builds its two sequence input as such:

In [59]:
max_length = 512 #max length for bert can be up to 512
batch_size = 6 #recommended batch size for bert is 16,32,... how ever on 
               #this dataset we are overfitting quiet often and smaller batch size works like regularization.
               #You might play with adding another dropout layer instead.

Encode Training and Test Dataset


Now lets combine the whole encoding process to one function;` so that we can map over our train and test data`.  

In [60]:
def convert_example_to_feature(review):   #why is it needed to do so? what if we did not use encode_plus? YOU HAVE TO DO ALL YOU DID TO TEST SENTENCE THE SAME TO ALL YOUR DATASET.
  return tokenizer.encode_plus(review,
                               max_length=max_length,
                               add_special_tokens=True,
                               pad_to_max_length=True,
                               return_attention_mask=True
)

In [61]:
print(bert_input)
# Bert input is including input_ids, token_type_ids , attention_mask; as presented below they are all numbers which which should be turned into dictionaries so that can be finally turned into 
#lists and by using append all the slices shoul b eturned into one sequence.

{'input_ids': [101, 3231, 19204, 3989, 6251, 2011, 29393, 7911, 2078, 1012, 2628, 2011, 2178, 6251, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


`BEFORE ENCODING IT IS NEEDED TO MAKE A DICTIONARY LIST`

In [62]:
# lets map to the expected input to TFBertForSequenceClassification, see here
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return{
      "input_ids": input_ids,
      "token_type_ids":token_type_ids,
      "attention_mask":attention_masks,

  }, label

    # what are these limitations?
def encode_examples(ds, limit=-1):
    #preparing a list of all features of Bert_input so that we can build up final 
    #tensorflow dataset from slices.
    input_ids_list = []
    token_type_ids_list= []
    attention_mask_list= []
    label_list = []
    # what are these limitations?
    if(limit > 0):
      ds=ds.take(limit)


    for review, label in tfds.as_numpy(ds):

        bert_input = convert_example_to_feature(review.decode())

        input_ids_list.append(bert_input["input_ids"])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list,attention_mask_list,token_type_ids_list,label_list)).map(map_example_to_dict)

In [63]:
ds_train_encoded = encode_examples(ds_train).shuffle(10000).batch(batch_size)
ds_test_encoded = encode_examples(ds_test).batch(batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [64]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

model =TFBertForSequenceClassification.from_pretrained('bert-base-uncased')


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_189']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [72]:
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True)
metrics =tf.keras.metrics.TopKCategoricalAccuracy(k=1)
optimizer = 'adam'
model.compile(optimizer , loss , metrics=[metrics])

In [75]:
model.summary()

Model: "tf_bert_for_sequence_classification_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_189 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
learning_rate = 2e-5
number_of_epoches =1
bert_history = model.fit(ds_train_encoded, epochs=number_of_epoches,validation_data = ds_test_encoded)